<a href="https://colab.research.google.com/github/arjunkpreman/Transfer-Learning-Project/blob/main/VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load pre-trained VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [6]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [7]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # num_classes is the number of classes in your task

In [8]:

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Set up data generators for training and validation
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg19.preprocess_input)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 3015 images belonging to 10 classes.


In [11]:
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg19.preprocess_input)
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 921 images belonging to 10 classes.


In [12]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    epochs=5
)

Epoch 1/5
94/94 [==============================] - 877s 9s/step - loss: 1.1822 - accuracy: 0.6416 - val_loss: 0.7351 - val_accuracy: 0.7545
Epoch 2/5
94/94 [==============================] - 21s 223ms/step - loss: 0.3594 - accuracy: 0.8763 - val_loss: 0.4904 - val_accuracy: 0.8248
Epoch 3/5
94/94 [==============================] - 21s 221ms/step - loss: 0.2141 - accuracy: 0.9363 - val_loss: 0.3970 - val_accuracy: 0.8616
Epoch 4/5
94/94 [==============================] - 21s 218ms/step - loss: 0.1559 - accuracy: 0.9511 - val_loss: 0.5219 - val_accuracy: 0.8292
Epoch 5/5
94/94 [==============================] - 21s 221ms/step - loss: 0.0930 - accuracy: 0.9806 - val_loss: 0.4111 - val_accuracy: 0.8560


In [13]:
# Save the trained model
model.save('/content/drive/MyDrive/Dataset/VGG19.h5')

In [14]:

model.evaluate(val_generator)

29/29 [==============================] - 8s 289ms/step - loss: 0.4057 - accuracy: 0.8578


[0.4057188630104065, 0.8577632904052734]